# Purpose

This notebook generates codes 

**DEPENDENCIES**<br>
`opentrons`
... This module only works in python 3.7 not in 3.11.<br>

<br>
** Roles **
sample_list: A list of Sample objects which encodes the concentrations of N chemical species<br>
... This keeps track of the assigned ** well plate locations**.<br>
run_files: A pickle file that Otto reads<br>
run.csv (-> labware_configuration.csv): A csv file that lists the positions of the tips etc.<br>
... Users MUST check before submitting the job.<br>

**File architecture**
/run.csv<br>
/samples: Location of  sample_list lives<br>
/prepared_samples: Location reserved for the sample_lists that were prepared by Otto<br>
... <br>

<br>

**Procedure**

0. CREATE A VIRTUAL ENVIRONMENT FOR `opentrons`. `conda create -n "otto" python==3.7`, then activate the environment `conda activate otto`
1. Create sample_list
... sample_list = [Sample1, Sample2, ...]
...... Sample 1 = {'peg': c in g/L, ' bsa': c in g/L, 'kcl': c in mM, ...}

2. Make a run.csv -> labware_configration.csv
... Configure the positions of well plates, tips, and falcon tubes (stock solutions).

3. Sync the folder to the server (Otto)
... sync run.csv, samples/*, runfiles/run_file*
... Specify the starting tip location.

4. Run the following command on the terminal. (Optional but recommended)
`opentrons_simulate SamplePrepCode_Fast.py > out.txt`
... out.txt contains all print statements in the code.
... This creates `SamplePrepSummary_DATE.csv` which lists the well plate numbers, etc.

5. Run SamplePrepCode_Fast.py (-> SamplePrepCode.py)
... Use the `nohup` command. (recommended)
... This step initiates the sample preparation. 
... The following command must be run on Terminal after connecting to Otto via ssh.

`nohup opentrons_execute /var/lib/jupyter/notebooks/otto_sample_prep/SamplePrepCode_Fast.py &`

... For Mac/Linux users, you can run this code from the notebook.

...... When a sample is prepared, a sample_list is moved to `prepared_samples`

6. Sync the Otto using scp (Necessary if we didn't run `opentrons_simulate SamplePrepCode_Fast.py > out.txt`.) 

... Grab `prepared_samples`,  `SamplePrepSummary_DATE.csv`, `out.txt`. 


# Import Modules

In [2]:
import definitions as otto # definitions is not an informative name. Also, def is a built-in word in Python. Avoid.
import sys
import opentrons
import ast
import os
from pathlib import Path

import softliv.softliv.labquest as lab
lab.make_solution(15, 2000, 84.007)

Recipe for 2000 mM ATP solution:
... Mix 2.52021 g = 2520.210 mg in  15.000 mL of solvent.


2.52021

# Define Functions

In [ ]:
def convert_sample_lists_to_df(vol_sample_per_dil, Num_dilutions, dilution_factor, sample_lists):
    data_list = []
    sample_number = 0
    sample_id_number = 0
    for sample_list in sample_lists:
        with open(master_dir / 'prepared_samples' / sample_list, 'rb') as f:
            metadata_sample, chemical_viscosity, samples = pickle.load(f)
        for ind,sample in enumerate(samples):
            sample_id_number += 1          # remove this later
            for i in range(Num_dilutions):
                sample_number += 1
                dil_well_loc = generate_unique_well_position(sample_number=sample_number,method='all',starts_with='F1')
                data_list.append([sample_id_number,sample.plate_number,sample.well, sample.sample_volume,sample.composition,dil_well_loc,dilution_factor,Num_dilutions,vol_sample_per_dil,(vol_sample_per_dil-1)*dilution_factor])   
        
    df = pd.DataFrame(data_list,columns=['Sample_ID','Plate','Well','Sample Volume','Composition','Dilution_Well','Dilution_Factor','Num_Dilutions','SampleVolumeDilution','WaterVolumeDilution'])
    return df
            

# File Architecture

"Firstly set the directory you want to work in - this is the folder where all the code will be saved."

**Takumi: You treat the working directory and the module directory the same. That is not recommended. If you do, you MUST have a feature to keep the code directory clean. **

In [25]:
# Working directory
# src = current_directory = os.getcwd()
src = current_directory = "~/Desktop"

# Directory on Otto
dst = 'root@10.49.35.97:/var/lib/jupyter/notebooks/'
directory_name = os.path.basename(src)

In [ ]:
# STEPS

# STEP 1: Insert info. about stock solutions

Firstly set the directory you want to work in - this is the folder where all the code will be saved.

Before creating Sample lists you must confirm that the chemical you want to add is in the list of chemicals in 'definitions.py'. If is not just add another line to it. The key is the name of the chemical you want to include. The value is a string that can either be 'water', 'bsa', or 'peg' and the idea is that these are the three kinds of viscosities you might encounter - 'water' for non-viscous samples, 'bsa' for intermediate viscosities and 'peg' for super viscous samples.

Also create the stock dictionary for all the stock concentrations you intend to use for this sample prep - with the exception of BSA and PEG everything else is in mM

In [3]:
# STOCK STOLUTIONS
stock_dict = {'bsa': 374,       # g/L
              'peg': 600,       # g/L
              'kp': 2000,        # mM
              'kcl': 4000,
#               'nh3':500,
#               'hco3':1000,
              'water':0,
              'nacl': 2000,
              'nascn': 2000,
              'mgcl2': 2000,
              'cacl2': 2000,
              'proline':2700,
              'lysine':1000,
#               'glut_acid':2000,
#               'adenine':1000,
              'glycine':1000,
              'glucose':1000,
              'nh4cl': 500,
              'nh4oh': 1000,
#               'h3po3':1000,
#               'hexanediol':1000,
#               'glucose':1000
            }

# Define 
chemical_list_and_relviscosity = {}
for key in stock_dict:
    chemical_list_and_relviscosity[key] = 'water'
chemical_list_and_relviscosity['bsa'] = 'bsa'   
chemical_list_and_relviscosity['peg'] = 'peg'


# Creating Sample Lists for Otto Process 

1. Add samples to the sample list either manually (if the sample list is simple with only one varying concentration and other chemicals are kept fixed) or using csv file. For csv file

In [79]:
# for manual addition of samples
otto.make_sample_list_manually(chemical_list_and_relviscosity, stock_dict=stock_dict)

list of chemicals: dict_keys(['bsa', 'peg', 'kp', 'kcl', 'water', 'nacl', 'nascn', 'mgcl2', 'cacl2', 'proline', 'lysine', 'glycine', 'glucose', 'nh4cl', 'nh4oh'])
make default sample containing bsa,peg,kp,kcl? (y/n)y
chemical for variation {}lysine
set conc of bsa in mM or g/L (for bsa and peg)46.6
set conc of peg in mM or g/L (for bsa and peg)226
set conc of kp in mM or g/L (for bsa and peg)100
set conc of kcl in mM or g/L (for bsa and peg)200
list of concs for variation of lysine in mM or g/L (for bsa and peg)0 50 100 150 200 250 300 400 500 600 700
max conc allowed is [390.19047619047615]
list of concs for variation of lysine in mM or g/L (for bsa and peg)0 50 100 150 200 250 300 350
list the order of mixing the chemicals in your sample - ['bsa', 'peg', 'kp', 'kcl', 'lysine', 'water']water kp kcl lysine bsa peg
do you want to make background samples? (y/n)n
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'lysine': 0.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl

In [68]:
# # under construction
# otto.make_sample_list_csv(filename='',chemical_list_and_relviscosity)              # for automatic addition of samples from csv file



SyntaxError: positional argument follows keyword argument (2993090432.py, line 1)

**Check the list of samples `sample_list` in `/samples`
... Each sample is numbered.

In [46]:
from importlib import reload
reload(otto)
otto.view_sample_lists()                 # for viewing the sample list


Sample list 0:
sample list: proline variation 
time: 2024-03-16 21:16:06
comments: 
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 0.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 50.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 100.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 150.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 200.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 300.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 400.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 500.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 600.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 700.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 800

3. **Estimate requirements for preparing samples** - this will tell you the minimum stock vols, pipette tips and pipettes and plates you need for preparing the samples.
    The function takes as input volume of sample, list of sample list numbers you want to run (for eg if you only want to run sample list 0 and 1 (numbers according to what is displayed in view_sample_list - then pass [0,1])) and dictionary of stocks
    

In [55]:
otto.estimate_requirements(1600, sample_list_nos=[1, 2, 3, 4], stock_dict=stock_dict)       # for estimating the requirements of the samples

# Check pipette
# Pipette tip types: p20, p300, p1000- Otto has two slots. Their default tip types are p300, p1000.
# 
# 11 stations. - divided into either stock 'cells' or well plates.
## Each cell can house 15mL x 6, 50mL x 4 falcon tubes.
### A wellplate- 6 x 16 wells


summary of the minimum requirements:
number of plates: 1.0
pipettes: ['p20', 'p300']
tip boxes: [1.0, 2.0]
Number of tips used: [2.0, 140.0]
minimum stock volumes required:
bsa: 4.369816849816851 mL
peg: 9.642666666666667 mL
kp: 1.28 mL
kcl: 1.28 mL
water: 4.547516483516485 mL
nacl: 0.0 mL
nascn: 0.0 mL
mgcl2: 0.0 mL
cacl2: 0.0 mL
proline: 0.0 mL
lysine: 0.0 mL
glycine: 2.24 mL
glucose: 0.0 mL
nh4cl: 0.0 mL
nh4oh: 2.24 mL


4. **Create run file for Otto** which will contain all the run information needed for the OT-2
This also creates a csv file called 'run.csv' which includes all the information of deck location and stock volumes. It is already pre-filled according to the estimated requirements but can be changed depending on the user's preference
Again - the inputs are the same as that for estimate_requirements

In [3]:
otto.create_run_file(1600, sample_list_nos=[1, 2, 3, 4, 6], stock_dict=stock_dict)      # for creating the run file

[419. 579. 271.]
Enter the chemicals that need not be mixed immediately?kp kcl water
run file saved as run_file-2024-03-19__14-37-09.pkl


This is the last step for the sample creation - now you only need to set up all the physical requirements for the sample prep and run the code for Otto! 

In [27]:
!open run.csv

'open' is not recognized as an internal or external command,
operable program or batch file.


## Running samples for Otto:

First, you need to update the run.csv file in the folder with all the stock volumes and the correct deck locations for all the labware you will be using
Once you are convinced of that, send all the files to otto's server:

In [20]:
# RUN THIS ON TERMINAL

('C:\\Users\\tm688\\Documents\\git\\softliv\\otto_sample_prep',
 'root@10.49.35.97:/var/lib/jupyter/notebooks/')

In [35]:
# SEND FILES
import os
src = os.getcwd()
dst = 'root@10.49.35.97:/var/lib/jupyter/notebooks/'
directory_name = os.path.basename(src)
# ot2_ssh_key should be stored in ~/.ssh/ot2_ssh_key (Mac/Linux)
#
# REMOVE FILES FROM PREVIOUS EXPERIMENTS
!ssh -i ~/.ssh/ot2_ssh_key root@10.49.35.97 "rm -r /var/lib/jupyter/notebooks/{directory_name}" exit
# UPLOAD FILES TO OTTO
# MAC/Linux
!scp -O -r -i ~/.ssh/ot2_ssh_key {src} {dst}

# END A SSH SESSION
# !exit

# NOTIFY USERS
os.system('say Otto has completed the sample preparation.')
print('Done')

directory_name

Done


unknown option -- O
usage: scp [-346ABCpqrTv] [-c cipher] [-F ssh_config] [-i identity_file]
            [-J destination] [-l limit] [-o ssh_option] [-P port]
            [-S program] source ... target


'otto_sample_prep'

In [36]:
# Windows
# SEND FILES
import os
src = os.getcwd()
dst = 'root@10.49.35.97:/var/lib/jupyter/notebooks/'
directory_name = os.path.basename(src)
# ot2_ssh_key should be stored in ~/.ssh/ot2_ssh_key (Mac/Linux)
#
# REMOVE FILES FROM PREVIOUS EXPERIMENTS
# !ssh -i ~/.ssh/ot2_ssh_key root@10.49.35.97 "rm -r /var/lib/jupyter/notebooks/{directory_name}" 
# UPLOAD FILES TO OTTO
# MAC/Linux
!scp -i ~/.ssh/ot2_ssh_key -r {src} {dst}

# END A SSH SESSION
# !exit

# NOTIFY USERS
os.system('say Otto is ready to run.')
print('Done')

directory_name

Done


'otto_sample_prep'

In [ ]:
# RUN THIS BKG JOB ON TERMINAL
ssh -i ~/.ssh/ot2_ssh_key root@10.49.35.97
nohup opentrons_execute /var/lib/jupyter/notebooks/otto_sample_prep/SamplePrepCode_Fast.py &

In [6]:

!opentrons_simulate SamplePrepCode_Fast.py > out.txt



C:\Users\tm688\.opentrons\robot_settings.json not found. Loading defaults
Deck calibration not found.
C:\Users\tm688\.opentrons\deck_calibration.json not found. Loading defaults


In [4]:
# EXECUTE


# run the code in the server once all the hardware has been set up
# don't use this yet!
!opentrons_simulate SamplePrepCode_Fast.py > out.txt
# !ssh -i ot2_ssh_key root@10.49.35.97
# EXECUTE
!ssh -i ~/.ssh/ot2_ssh_key root@10.49.35.97 'sh -l -c "nohup opentrons_simulate /var/lib/jupyter/notebooks/otto_sample_prep/SamplePrepCode_Fast.py &"'
# RUN THIS BKG JOB ON TERMINAL
# nohup opentrons_execute /var/lib/jupyter/notebooks/{directory_name}/SamplePrepCode_Fast.py &

os.system('say Otto has completed the sample preparation.')





sh: sh -l -c nohup opentrons_simulate /var/lib/jupyter/notebooks/otto_sample_prep/SamplePrepCode_Fast.py &: not found


1

In [11]:
otto.view_sample_lists()

Sample list 0:
sample list: proline variation 
time: 2024-03-16 21:16:06
comments: 
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 0.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 50.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 100.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 150.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 200.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 300.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 400.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 500.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 600.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 700.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'proline': 800

In [10]:
# Retrieve the summary file once the sample prep is done:
# Kaarthik's original code (TM: This does not look right)
## Mac/Linux
# !scp -O -r -i ~/.ssh/ot2_ssh_key root@10.49.35.97:/var/lib/jupyter/notebooks/{directory_name}/prepared_samples {current_directory}
## Windows
src = 'root@10.49.35.97:/var/lib/jupyter/notebooks/otto_sample_prep/prepared_samples'
!scp -i ~/.ssh/ot2_ssh_key -r {src} {current_directory}

# Seperating dilute from dense phase and performing dilutions on the dilute phase:

In [22]:
# view all the prepared samples
otto.view_sample_lists(dir = Path(current_directory) / 'prepared_samples/')

Sample list 0:
sample list: glycine variation
time: 2024-03-16 21:19:42
comments: 
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'glycine': 0.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'glycine': 50.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'glycine': 100.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'glycine': 150.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'glycine': 200.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'glycine': 250.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'glycine': 300.0, 'water': 0}
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'glycine': 350.0, 'water': 0}
order of mixing: ['water', 'kp', 'kcl', 'glycine', 'bsa', 'peg']



Sample list 1:
sample list: nh4oh variation
time: 2024-03-16 21:21:55
comments: 
{'bsa': 46.6, 'peg': 226.0, 'kp': 100.0, 'kcl': 200.0, 'nh4oh': 0.0, 'water': 0}
{'bsa': 46

In [47]:

create_run_file_for_dilutions(sample_list_nos=[3],vol_sample_per_dil=30,dilution_factor=10,Num_dilutions=2,stock_dict=stock_dict,vol_fraction_dense=0.2)



summary of the minimum requirements:
pipettes: ['p300', 'p1000']
tip boxes: [1.0, 1.0]
Number of tips used: [24.0, 6.0]
run file saved as dilutions_run_file-2024-03-06__15-27-19.pkl


In [48]:
!open dilution_labware.csv

In [49]:
current_directory = os.getcwd()
directory_name = os.path.basename(current_directory)
!ssh -i ~/.ssh/ot2_ssh_key root@10.49.35.97 "rm -r /var/lib/jupyter/notebooks/{directory_name}"
!scp -O -r -i ~/.ssh/ot2_ssh_key {current_directory} root@10.49.35.97:/var/lib/jupyter/notebooks/

OttoFunctions.py                              100%   23KB 863.4KB/s   00:00    
SamplePrepCode.py                             100% 3316   292.4KB/s   00:00    
Dilutions.py                                  100% 4020   259.0KB/s   00:00    
out.txt                                       100%  444KB 941.3KB/s   00:00    
.DS_Store                                     100%   10KB 795.4KB/s   00:00    
definitions.py                                100%   52KB   1.0MB/s   00:00    
SamplePrepCode_Fast.py                        100% 3806   354.7KB/s   00:00    
CreatingSampleLists.ipynb                     100%   17KB 441.0KB/s   00:00    
sample_list-2024-03-04__14-46-12.pkl          100% 1681   165.6KB/s   00:00    
sample_list-2024-03-04__14-47-56.pkl          100% 1681   162.1KB/s   00:00    
sample_list-2024-03-04__14-45-38.pkl          100% 1675   143.3KB/s   00:00    
sample_list-2024-03-04__14-47-00.pkl          100% 1683   159.2KB/s   00:00    
run.csv                                 

In [ ]:
!ssh -i ~/.ssh/ot2_ssh_key root@10.49.35.97 'sh -l -c "nohup opentrons_execute /var/lib/jupyter/notebooks/{directory_name}/BSA_dilutions.py"'

In [27]:
# transfer the dilute phase of samples to new plate:

first_run_file = sorted([f for f in os.listdir(master_dir) if f.startswith('dilutions_run_file')])[0]
print(first_run_file)
with open(master_dir / first_run_file,'rb') as f:
    vol_sample_per_dil,Num_dilutions,dilution_factor,sample_lists = pickle.load(f)

df = convert_sample_lists_to_df(vol_sample_per_dil,Num_dilutions,dilution_factor,sample_lists)
for sample_id, group in df.groupby('Sample_ID'):
    row = group.iloc[0]
    sample_ID = row['Sample_ID']
    print(row['Sample_ID'],row['Well'])

dilutions_run_file-2024-03-05__17-18-05.pkl
1 A9
2 A10
3 A11
4 A12
5 B1
6 B2
7 B3
8 B4
9 B5
10 B6
11 B7
12 B8
13 B9
14 B10
15 B11
16 B12
17 C1
18 C2
